In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Small, fast BERT-based model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
def create_context_from_csv(df):
    row_texts = []
    for _, row in df.iterrows():
        sentence = sentence = f"Student_ID {row['Student_ID']}: {row['Name']} is {row['Age']} years old, enrolled in {row['Course']}, scored {row['Marks']} marks, and got grade {row['Grade']}."
        row_texts.append(sentence)
    return row_texts

def get_best_match_answer(question, row_texts):
    # Encode both question and all rows
    question_embedding = model.encode(question, convert_to_tensor=True)
    row_embeddings = model.encode(row_texts, convert_to_tensor=True)

    # Compute cosine similarity
    scores = util.pytorch_cos_sim(question_embedding, row_embeddings)[0]
    top_idx = scores.argmax().item()
    best_score = scores[top_idx].item()

    # Optional: filter out low-confidence matches
    if best_score < 0.4:
        return "🤖 Sorry, I couldn not find a confident match. Please rephrase your question."

    return row_texts[top_idx]

In [5]:
print("BERT-Powered Semantic Student Chatbot")
csv_path = r"/content/DL.xlsx"
df = pd.read_excel(csv_path)
row_texts = create_context_from_csv(df)
while True:
    question = input(" Your Question: ")
    if question.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    answer = get_best_match_answer(question, row_texts)
    print(f"Answer: {answer}\n")

BERT-Powered Semantic Student Chatbot
 Your Question: What is the age of Mani?
Answer: Student_ID 101: Mani is 10 years old, enrolled in maths, scored 46 marks, and got grade B.

 Your Question: What is the course maxwell enrolled?
Answer: Student_ID 105: maxwell is 12 years old, enrolled in english, scored 46 marks, and got grade B.

 Your Question: exit
Goodbye!
